#Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_pinball_loss, d2_pinball_score

from sklearn.model_selection import train_test_split, cross_val_score, validation_curve
import os
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.preprocessing import StandardScaler
import time
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import log_loss, d2_pinball_score, get_scorer_names
from sklearn.utils import resample

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score, median_absolute_error, r2_score

import math

import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error, mean_pinball_loss, make_scorer, accuracy_score, balanced_accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support

from google.colab import drive
from os import listdir
from os.path import isfile, join
from joblib import dump, load


from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV, LeaveOneOut
from pprint import pprint


drive.mount('/content/drive')

PATH = '/content/drive/MyDrive/nda'

Mounted at /content/drive


#Import Datasets

In [ ]:
new_data_path = PATH + '/new_data'
files = [f for f in listdir(new_data_path) if isfile(join(new_data_path, f))]
print(files)


data_g = pd.read_csv(join(new_data_path,files[0]))
data_e = pd.read_csv(join(new_data_path, files[1]))

['GermanDataset.csv', 'EuropeanDataset.csv']


#Split Data

In [ ]:
def train_test_split2(X, y, test=0.2, ySplitUnravel = True):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test, random_state=4, shuffle=True, stratify=y['MF'])

  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  if ySplitUnravel == True:
    y_train_snr = y_train.iloc[:,0].to_numpy().ravel()
    y_train_mf =  y_train.iloc[:,1].to_numpy().ravel()

    y_test_snr = y_test.iloc[:,0].to_numpy().ravel()
    y_test_mf =  y_test.iloc[:,1].to_numpy().ravel()

    return X_train, X_test, y_train_snr, y_train_mf, y_test_snr, y_test_mf
  else:
    return X_train, X_test, y_train, y_test

# Original datasets, split into train/test/validation including seperate target vectors (snr and mf)
G_train, G_test, g_train_snr, g_train_mf, g_test_snr, g_test_mf = train_test_split2(data_g.loc[:, ~data_g.columns.isin(['SNR', 'MF'])], data_g[['SNR', 'MF']], test=0.2)
E_train, E_test, e_train_snr, e_train_mf, e_test_snr, e_test_mf = train_test_split2(data_e.loc[:, ~data_e.columns.isin(['SNR', 'MF'])], data_e[['SNR', 'MF']], test=0.2)



In [ ]:
e_train_mf = [str(i) for i in e_train_mf]
g_train_mf = [str(i) for i in g_train_mf]

e_test_mf = [str(i) for i in e_test_mf]
g_test_mf = [str(i) for i in g_test_mf]


In [ ]:
# Saving the split for future use

split_data_path = PATH + '/split_data/'
np.savetxt(join(split_data_path, 'E_train.out'), E_train, delimiter=',')
np.savetxt(join(split_data_path,'G_train.out'), G_train, delimiter=',')


np.savetxt(join(split_data_path,'E_test.out'), E_test, delimiter=',')
np.savetxt(join(split_data_path,'G_test.out'), G_test, delimiter=',')


np.savetxt(join(split_data_path,'e_train_snr.out'), e_train_snr, delimiter=',')
np.savetxt(join(split_data_path,'g_train_snr.out'), g_train_snr, delimiter=',')
np.savetxt(join(split_data_path,'e_test_snr.out'), e_test_snr, delimiter=',')
np.savetxt(join(split_data_path,'g_test_snr.out'), g_test_snr, delimiter=',')

In [ ]:
# SNR to MF Function

boundries = [8.7, 12.8, 15.2, 18.2, 21.0]

def SNR_to_MF(SNR):
    epsilon = 1e-6  # Small epsilon value for comparison

    if SNR < boundries[0] - epsilon:
        return 0

    elif (boundries[0] - epsilon <= SNR) and (SNR < boundries[1]):
        return 4

    elif (boundries[1] <= SNR) and (SNR < boundries[2]):
        return 8

    elif (boundries[2] <= SNR) and (SNR < boundries[3]):
        return 16

    elif (boundries[3] <= SNR) and (SNR < boundries[4]):
        return 32

    else:  # SNR >= 23.7:
        return 64


#Helper Functions to Evaluate Regressor and Classifier Models

In [ ]:
def eval_model_classifier(y_true1, y_pred1):

  y_true = np.array(y_true1, dtype=float)
  y_pred = np.array(y_pred1, dtype=float)
  accuracy = accuracy_score(y_true, y_pred)
  precision, recall, fscore, support = precision_recall_fscore_support(y_true, y_pred, zero_division = 0)
  under = np.sum(y_true > y_pred)
  over = np.sum(y_true < y_pred)
  correct = np.sum(y_true == y_pred)
  miss_rate = (over+under)/ (correct+over+under)

  results = {'Accuracy': accuracy,
             'Precision': precision.mean(),
             'Recall': recall.mean(),
             'FScore': fscore.mean(),
             'Under Allocations': under,
             'Over Allocations': over,
             'Correct Allocations': correct,
             'Miss Rate': miss_rate}

  return results

In [ ]:
PENALIZE_OVER  = 0
PENALIZE_UNDER = 1

def eval_model(y_true, y_pred, alpha=-1):

  mse = mean_squared_error(y_true, y_pred)
  pbl = mean_pinball_loss(y_true, y_pred, alpha=0.5)
  pbo = mean_pinball_loss(y_true, y_pred, alpha=PENALIZE_OVER)
  pbu = mean_pinball_loss(y_true, y_pred, alpha=PENALIZE_UNDER)
  pba = mean_pinball_loss(y_true, y_pred, alpha=alpha)
  mae = mean_absolute_error(y_true, y_pred)
  r2 = r2_score(y_true, y_pred)
  d2 = d2_pinball_score(y_true, y_pred)
  aic, bic = aic_bic(y_true, y_pred, alpha)
  mf_true = np.array([SNR_to_MF(x) for x in y_true])
  mf_pred = np.array([SNR_to_MF(x) for x in y_pred])
  under = np.sum(mf_true > mf_pred)
  over = np.sum(mf_true < mf_pred)
  correct = np.sum(mf_true == mf_pred)
  miss_rate = (over+under)/ (correct+over+under)

  results = {'Mean Squared Error':mse,
             'Mean Pinball Loss': pbl,
             'Pinball Loss Penalize Under': pbu,
             'Pinball Loss Penalize Over': pbo,
             'Pinball Loss At Alpha': pba,
             'Mean Absolute Error': mae,
             'R2 Square': r2,
             'D2 Pinball': d2,
             'AIC': aic,
             'BIC': bic,
             'Under Allocations': under,
             'Over Allocations': over,
             'Correct Allocations': correct,
             'Miss Rate': miss_rate}

  return results


#Baseline Models
Straight out of the box; using default parameters

In [ ]:
def gen_default_models(X, y_mf, y_snr, file_suffix):
  print('******************************************************************')
  print(f'Classifier with default parameters')
  clf_def = GradientBoostingClassifier()
  clf_def_cvscore = cross_val_score(clf_def, X, y_mf)
  print(f'\tCV Score:\t\t{clf_def_cvscore.mean():3.6}')
  t0 = time.time()
  clf_def.fit(X, y_mf)
  t1 = time.time()
  dur = t1-t0
  print(f'\tTraining Duration:\t{dur:3.5} s')
  clf_def_hat = clf_def.predict(X)
  precision, recall, fscore, support = precision_recall_fscore_support(y_mf, clf_def_hat, zero_division=0)
  res = eval_model_classifier(y_mf, clf_def_hat)
  res['Cross Validation'] = clf_def_cvscore.mean()
  metric = {}
  metric['Clf_Def_'+ file_suffix] = res
  metric_df = pd.DataFrame.from_dict(metric, orient='index')
  def_metrics_filepath = 'metrics/all_metrics_clfdef_' + file_suffix + 'train.csv'
  metric_df.to_csv(join(PATH, def_metrics_filepath))
  #np.savetxt(join(PATH, def_metrics_filepath), metric_df, delimiter=',')

  print('******************************************************************')
  print(f'Quantile Regressor with default parameters')

  q5_def = GradientBoostingRegressor(loss='quantile')
  pinballscorer = make_scorer(mean_pinball_loss, alpha=0.5, greater_is_better=False)

  q5_def_cvscore = -cross_val_score(q5_def, X, y_snr, scoring=pinballscorer)
  print(f'\tCV Loss:\t\t{q5_def_cvscore.mean():3.6}')
  t0 = time.time()
  q5_def.fit(X, y_snr)
  t1 = time.time()
  dur = t1-t0
  print(f'\tTraining Duration:\t{dur:3.5} s')
  q5_def_hat = q5_def.predict(X)

  res1 = eval_model(y_snr, q5_def_hat, 0.5)
  res1['Cross Validation'] = q5_def_cvscore.mean()
  metric1 = {}
  metric1['D0.50_pb0.50_' + file_suffix] = res1
  metric_df1 = pd.DataFrame.from_dict(metric1, orient='index')
  def_metrics_filepath1 = 'metrics/all_metrics_q5def_' + file_suffix + 'train.csv'
  metric_df1.to_csv(join(PATH, def_metrics_filepath1))

  filename1 = 'models/def_classifier_' + file_suffix + '.joblib'
  filename2 = 'models/def_q5regressor_' + file_suffix + '.joblib'

  dump(clf_def, join(PATH, filename1))
  dump(q5_def, join(PATH, filename2))


print('Models trained over European Set')
gen_default_models(E_train, e_train_mf, e_train_snr, file_suffix='e')
print('Models trained over German Set')
gen_default_models(G_train, g_train_mf, g_train_snr, file_suffix='g')


Models trained over European Set
******************************************************************
Classifier with default parameters
	CV Score:		0.940614
	Training Duration:	0.92373 s
******************************************************************
Quantile Regressor with default parameters
	CV Loss:		0.16823
	Training Duration:	0.44652 s
Models trained over German Set
******************************************************************
Classifier with default parameters
	CV Score:		0.934926
	Training Duration:	1.1447 s
******************************************************************
Quantile Regressor with default parameters
	CV Loss:		0.136345
	Training Duration:	0.44091 s


#Fine-Tined Models:
Hyperparameter optimization

#Generate Classifier Models

In [ ]:
def optimize_classifier(X_train, y_train, space, n_trials):
    #***************************************************************************
    #********* Perform hyperparameter optimization via crossvalidation *********
    def hyperopt_train_test(params_):

      params = {
                  'learning_rate'     : params_['learning_rate'],
                  'n_estimators'      : int(params_['n_estimators']),
                  'max_depth'         : int(params_['max_depth']),
                  'min_samples_leaf'  : int(params_['min_samples_leaf']),
                  'min_samples_split' : int(params_['min_samples_split'])
                }
      model = GradientBoostingClassifier(loss='log_loss',**params, random_state=11)

      return cross_val_score(model, X_train, y_train, cv = 3)

    def f(params):
        acc = hyperopt_train_test(params)
        return {'loss': -acc.mean(), 'status': STATUS_OK, 'loss_variance': np.var(acc, ddof=1)}

    trials = Trials()

    #***************************************************************************
    #******** Print best hyperparameters obtained with crossvalidation *********
    ta = time.time()
    best_params = fmin(fn=f, space=space, algo=tpe.suggest, max_evals=n_trials, trials=trials)
    tb = time.time()

    best_params = hyperopt.space_eval(space, best_params)
    print('Best Params Found by Hyperopt:')
    print(f'{best_params}\n')

    #***************************************************************************
    #********** Print best cv accuracy (across all combos) & duration **********
    best_cv_acc = trials.best_trial['result']['loss']

    print(f'Best CV Accuracy During Hyperopt: {-round(best_cv_acc, 6)}')
    print(f'Cross Validation Duration: {round(tb-ta, 5)} s')

    #***************************************************************************
    #*********** Retrain a new model with best hyperparameters using ***********
    #*********** the entire training set (X_train, y_train) ********************
    best_params_ = {
                  'learning_rate'     : best_params['learning_rate'],
                  'n_estimators'      : int(best_params['n_estimators']),
                  'max_depth'         : int(best_params['max_depth']),
                  'min_samples_leaf'  : int(best_params['min_samples_leaf']),
                  'min_samples_split' : int(best_params['min_samples_split'])
                }
    model = GradientBoostingClassifier(loss='log_loss', **best_params_, random_state=11)

    t0 = time.time()
    model.fit(X_train, y_train)
    t1 = time.time()

    scr = model.score(X_train, y_train)
    print(f'Model Score: {scr}')
    #***************************************************************************
    #************* Print training results (train accuracy and duration) ********
    print(f'Training Duration: {round(t1 - t0, 6)} s\n')

    #***************************************************************************
    #************************ Return the trained model *************************
    trials_df = pd.DataFrame()
    for t in trials:
      params = pd.DataFrame(t['misc']['vals'])
      params['loss'] = t['result']['loss']
      params['loss_var'] = t['result']['loss_variance']
      trials_df = pd.concat([trials_df, params], ignore_index=True, axis=0)



    return model, best_cv_acc, trials_df


space = {
     'learning_rate': hp.uniform('learning_rate', 0.001, 0.9),
     'n_estimators' : hp.quniform('n_estimators', 50, 100, 5),
     'max_depth': hp.quniform('max_depth', 2, 10, 1),
     'min_samples_leaf': hp.quniform('min_samples_leaf', 3, 50, 1),
     'min_samples_split': hp.quniform('min_samples_split', 5, 30, 1)
    }

clf_opt_e, clf_opt_score_e, clf_trials_e = optimize_classifier(E_train, e_train_mf, space, n_trials=150)
dump(clf_opt_e, join(PATH,'models/opt150_classifier_e.joblib'))



clf_opt_g, clf_opt_score_g, clf_trials_g = optimize_classifier(G_train, g_train_mf, space, n_trials=150)
dump(clf_opt_g, join(PATH,'models/opt150_classifier_g.joblib'))





100%|██████████| 150/150 [05:13<00:00,  2.09s/trial, best loss: -0.9419845899404383]
Best Params Found by Hyperopt:
{'learning_rate': 0.24563060794923214, 'max_depth': 2.0, 'min_samples_leaf': 27.0, 'min_samples_split': 7.0, 'n_estimators': 65.0}

Best CV Accuracy During Hyperopt: 0.941985
Cross Validation Duration: 313.19005 s
Model Score: 0.9665529010238908
Training Duration: 0.48712 s

100%|██████████| 150/150 [05:04<00:00,  2.03s/trial, best loss: -0.9409163433600375]
Best Params Found by Hyperopt:
{'learning_rate': 0.33646423098832695, 'max_depth': 3.0, 'min_samples_leaf': 25.0, 'min_samples_split': 30.0, 'n_estimators': 55.0}

Best CV Accuracy During Hyperopt: 0.940916
Cross Validation Duration: 304.23635 s
Model Score: 0.9925205684367988
Training Duration: 0.501074 s



['/content/drive/MyDrive/nda/models/opt150_classifier_g.joblib']

#Add metrics for classifiers

In [ ]:
clf_opt_metrics_e = eval_model_classifier(e_train_mf, clf_opt_e.predict(E_train))
clf_opt_metrics_e['Cross Validation'] = clf_opt_score_e
clf_opt_metrics_g = eval_model_classifier(g_train_mf, clf_opt_g.predict(G_train))
clf_opt_metrics_g['Cross Validation'] = clf_opt_score_g

metrics_e = {}
metrics_e['Opt_Clf_e'] = clf_opt_metrics_e
metrics_g = {}
metrics_g['Opt_Clf_g'] = clf_opt_metrics_g


metric_df_e = pd.DataFrame.from_dict(metrics_e, orient='index')
def_metrics_filepath_e = 'metrics/all_metrics_clfopt_etrain.csv'
metric_df_e.to_csv(join(PATH, def_metrics_filepath_e))


metric_df_g = pd.DataFrame.from_dict(metrics_g, orient='index')
def_metrics_filepath_g = 'metrics/all_metrics_clfopt_gtrain.csv'
metric_df_g.to_csv(join(PATH, def_metrics_filepath_g))


#Fine Tune Quantile Regressor function

In [ ]:
def optimize_regressorpb(X_train, y_train, space, Q_alpha=0.5, L_alpha=0.5, n_trials=3):
    #***************************************************************************
    #********* Perform hyperparameter optimization via crossvalidation *********

    scoring_func=make_scorer(mean_pinball_loss, alpha=L_alpha, greater_is_better=False)

    def hyperopt_train_test(params_):
      params = {
                'learning_rate'     : params_['learning_rate'],
                'n_estimators'      : int(params_['n_estimators']),
                'max_depth'         : int(params_['max_depth']),
                'min_samples_leaf'  : int(params_['min_samples_leaf']),
                'min_samples_split' : int(params_['min_samples_split'])
                }
      model = GradientBoostingRegressor(loss='quantile',alpha=Q_alpha, **params, random_state=11)
      return cross_val_score(model, X_train, y_train, cv = 3, scoring=scoring_func)

    def f(space):
        losses = hyperopt_train_test(space)
        return {'loss': -losses.mean(), 'status': STATUS_OK,  'loss_variance': np.var(losses, ddof=1)}

    trials = Trials()
    #***************************************************************************
    #******** Print best hyperparameters obtained with crossvalidation *********
    ta = time.time()
    best_params = fmin(f, space, algo=tpe.suggest, max_evals=n_trials, trials=trials)
    tb = time.time()

    best_params = hyperopt.space_eval(space, best_params)
    print('Best Params Found by Hyperopt:')
    print(f'{best_params}\n')

    #***************************************************************************
    #********** Print best cv accuracy (across all combos) & duration **********
    best_cv_acc = trials.best_trial['result']['loss']

    print(f'\nBest CV Loss During Hyperopt: {round(best_cv_acc, 6)}')
    print(f'Cross Validation Duration: {round(tb-ta, 5)} s\n')

    #***************************************************************************
    #*********** Retrain a new model with best hyperparameters using ***********
    #*********** the entire training set (X_train, y_train) ********************
    best_params_ = {
                  'learning_rate'     : best_params['learning_rate'],
                  'n_estimators'      : int(best_params['n_estimators']),
                  'max_depth'         : int(best_params['max_depth']),
                  'min_samples_leaf'  : int(best_params['min_samples_leaf']),
                  'min_samples_split' : int(best_params['min_samples_split'])
                }

    model = GradientBoostingRegressor(loss='quantile',alpha=Q_alpha, **best_params_, random_state=11)

    t0 = time.time()
    model.fit(X_train, y_train)
    t1 = time.time()

    scr = model.score(X_train, y_train)
    print(f'Model Score: {scr}')
    #***************************************************************************
    #************* Print training results (train accuracy and duration) ********
    print(f'Training duration: {round(t1 - t0, 6)} s\n')
    #***************************************************************************
    #************************ Return the trained model *************************

    trials_df = pd.DataFrame()
    for t in trials:
      params = pd.DataFrame(t['misc']['vals'])
      params['loss'] = t['result']['loss']
      params['loss_var'] = t['result']['loss_variance']
      trials_df = pd.concat([trials_df, params], ignore_index=True, axis=0)

    return model, best_cv_acc, trials_df


In [ ]:
def aic_bic(y_true, y_pred, q, verbose=False):
  residuals = y_true - y_pred
  scale = np.percentile(np.abs(residuals), 100*q)
  log_likelihood_est = np.log(q / scale) - q * np.abs(residuals) / scale
  n = len(y_true)
  k = 5
  aic = -2 * np.sum(log_likelihood_est) + 2 * k
  bic = -2 * np.sum(log_likelihood_est) + np.log(n) * k
  if verbose:
    print(f'AIC: {aic}')
    print(f'BIC: {bic}')
  return aic, bic

#Generate Quantile Regressor Models
Note about the following function: be aware that it takes a long time to run to completion.

In [ ]:
def gen_opt_models(X, y, file_suffix, metric_suffix):
  all_models = {}
  all_metrics = {}
  all_predictions = pd.DataFrame()

  alphas = [0.1, 0.25, 0.5, 0.75, 0.9]
  for a in alphas:
    loss = [0, 1]
    loss.append(a)
    for b in loss:
      model_str = 'Q%.2f_pb%.2f_%c' % (a, b, file_suffix)
      print('********************************************************************************************************************************************')
      print('********************************************************************************************************************************************')
      print(f'Quantile: {a}\tPenalize: {b}\tModel Name: {model_str}')
      gbr, score, trials = optimize_regressorpb(X, y, space, Q_alpha=a, L_alpha=b, n_trials=150)
      filename = 'models/' + model_str + '.joblib'
      dump(gbr, join(PATH,filename))

      # Save also the predictions for graphing later...
      y_hat = gbr.predict(X)
      y_hat_mf = [SNR_to_MF(x) for x in y_hat]

      all_predictions[model_str+'_SNR'] = y_hat
      all_predictions[model_str+'_MF'] = y_hat_mf

      res = eval_model(y, y_hat, a)
      res['Cross Validation'] = score
      all_models[model_str] = gbr
      all_metrics[model_str] = res
  all_metrics = pd.DataFrame.from_dict(all_metrics, orient='index')
  metric_path = 'metrics/all_metrics_' + file_suffix + metric_suffix + '.csv'
  all_metrics.to_csv(join(PATH, metric_path))



gen_opt_models(E_train, e_train_snr, 'e', 'train')
gen_opt_models(G_train, g_train_snr, 'g', 'train')





********************************************************************************************************************************************
********************************************************************************************************************************************
Quantile: 0.1	Penalize: 0	Model Name: Q0.10_pb0.00_e
100%|██████████| 150/150 [03:36<00:00,  1.44s/trial, best loss: 0.008063658386763635]
Best Params Found by Hyperopt:
{'learning_rate': 0.03619496185239128, 'max_depth': 8.0, 'min_samples_leaf': 35.0, 'min_samples_split': 9.0, 'n_estimators': 60.0}


Best CV Loss During Hyperopt: 0.008064
Cross Validation Duration: 216.32977 s

Model Score: 0.2840677938788838
Training duration: 0.422531 s

********************************************************************************************************************************************
************************************************************************************************************************************